<a href="https://colab.research.google.com/github/dvarkless/InnopolisDS/blob/main/homeworks/BERT_fine_tuning_russian_kinopoisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment analysis on Kinopoisk movie reviews dataset using pretrained BERT model

Импорты: pytorch, bert, вспомогательные библиотеки ,шкала прогресса, отключение предупреждений

In [ ]:
import shutup; shutup.please()
from pathlib import Path
import numpy as np
import pandas as pd

# BERT imports
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from alive_progress import alive_bar


# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)

In [ ]:
DATA_DIR="../LSTM Sentiment/"
PROP = 0.2

Класс кастомного датасета позволит токенизировать текст по мере необходимости

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    if isinstance(idx, int):
        idx = idx if idx < len(self) else len(self)-1
    text = str(self.texts[idx])
    target = self.targets[idx]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
dataset_path = DATA_DIR / Path('dataset')

df = pd.DataFrame(columns=['review', 'sentiment'])

class_names = ["Negative", "Neutral", "Positive"]
class_names_converter = {
    'neg': 'Negative',
    'pos': 'Positive',
    'neu': 'Neutral',
}

def standardize_text(df, content_field):
    df[content_field] = df[content_field].str.replace(r"http\S+", "")
    df[content_field] = df[content_field].str.replace(r"@\S+", "")
    df[content_field] = df[content_field].str.replace(
        r"[^А-Яа-яA-Za-z0-9Ёё(),!?@\'\`\"\_\n]", " ")
    df[content_field] = df[content_field].str.replace(r"[Ёё]", "е")
    df[content_field] = df[content_field].str.replace(r"[\t\n]", "")
    df[content_field] = df[content_field].str.replace(r"[^А-Яа-яa-zA-Z]", " ")
    df[content_field] = df[content_field].str.lower()
    return df

def name_to_id(name):
    return class_names.index(name)

Загрузим датасет отзывов с кинопоиска в датафрейм Пандас
Для этого надо пройтись по каждой директории, открыть текстовый файл и загрузить его содержимое в датафрейм
Далее идет стандартизация текста и разбиение его на тестовую и проверочную выборки

In [ ]:
for class_path in dataset_path.iterdir():
    if class_path.is_dir():
        dirs = np.array(list(class_path.iterdir()))
        np.random.shuffle(dirs)
        rews_fhs = np.random.choice(dirs, round(len(dirs)))
        print(f'len = {rews_fhs.shape}')
        print(class_names_converter[class_path.name])
        for rew_fh in rews_fhs:
            with open(Path(rew_fh), encoding='utf-8') as f:
                review = f.read()
                current_df = pd.DataFrame(
                    {'review': [review], 'sentiment': class_names_converter[class_path.name]})
                df = pd.concat([df, current_df], ignore_index=True)

df = df.sample(frac=1).reset_index(drop=True)

len = (24704,)
Neutral
len = (87138,)
Positive
len = (19827,)
Negative


                                                   review sentiment
0       мне не совсем понятно  почему фильм имеет тако...  Positive
1       после распада советского союза родителям и мне...  Positive
2       может быть  кто то скажет  что это триллер или...  Positive
3       великолепный фильм  после просмотра остается м...  Positive
4       чат   средство обмена сообщениями по компьютер...  Positive
...                                                   ...       ...
131664  гилберт грейп живет в маленьком американском г...  Positive
131665  ах  какие слова  едва ли не любимейший мой мом...  Positive
131666  в последнее время почему то захотелось лучше п...  Positive
131667  фильм бесподобен  скажу сразу  если вы скучает...  Positive
131668   трасса     это фильм о том  к чему могут прив...  Positive

[131669 rows x 2 columns]

In [ ]:
df = standardize_text(df, "review")
df['sentiment'] = df['sentiment'].map(name_to_id)

In [ ]:
train_dataset, eval_dataset = train_test_split(df, test_size=PROP)

In [ ]:
train_dataset.to_csv('Kinopoisk_train.csv')
eval_dataset.to_csv('Kinopoisk_eval.csv')

Так как датасет был конвертирован в csv заранее, то пропустим выполнение строк выше и просто загрузим выборки из файла, который находится в локальной машине

In [ ]:
train_dataset = pd.read_csv('Kinopoisk_train.csv')
eval_dataset = pd.read_csv('Kinopoisk_eval.csv')

Инициируем токенайзер, модель и оборачиваем датасет в кастомный класс
Используем модель rubert-tiny2, тк она лучше всего подходит для задач NLP на русском языке

In [ ]:
rubert_path = 'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(rubert_path)
 

In [ ]:
train_dataset = CustomDataset(train_dataset['review'], train_dataset['sentiment'], tokenizer, max_len=512)
eval_dataset = CustomDataset(eval_dataset['review'], eval_dataset['sentiment'], tokenizer, max_len=512)

In [ ]:
n_classes = 3

model = BertForSequenceClassification.from_pretrained(rubert_path)

out_features = model.bert.encoder.layer[1].output.dense.out_features
model.classifier = torch.nn.Linear(out_features, n_classes)

После определения модели можно определить оптимизатор, функцию потерь
и класс, который будет обучать и проверять модель

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
class BertClassifier:
    def __init__(self, model, tokenizer, optimizer, loss, n_classes=3, max_len=512, model_save_path='rubert_on_kinopoisk.pt', log=True):
        self.model = model
        self.tokenizer = tokenizer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.optimizer = optimizer
        self.loss = loss
        self.loss.to(self.device)
        self.model_save_path=model_save_path
        self.max_len = max_len
        self.log = log
        self.model.to(self.device)
    
    def fit(self, train, eval, epochs=2, save_model=True):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0
        best_accuracy = 0


        train_dl = DataLoader(train, batch_size=16, shuffle=True)
        scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_dl) * epochs
            )

        with alive_bar(epochs*len(train)//16, title=f'Обучение модели ruBERT', force_tty=True, bar='filling') as bar:
            for epoch in range(epochs):

                correct_predictions = 0
                for data in train_dl:

                    input_ids = data["input_ids"].to(self.device)
                    attention_mask = data["attention_mask"].to(self.device)
                    targets = data["targets"].to(self.device)

                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask
                        )

                    preds = torch.argmax(outputs.logits, dim=1)
                    loss = self.loss(outputs.logits, targets)

                    correct_predictions += torch.sum(preds == targets)

                    losses.append(loss.item())
                    
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    self.optimizer.step()
                    scheduler.step()
                    self.optimizer.zero_grad()
                    bar()

                train_acc = correct_predictions.double() / len(train)
                train_loss = np.mean(losses)

                eval_acc, eval_loss = self.eval(eval)
                
                if self.log:
                    print(f'===========Epoch {epoch+1}/{epochs}===========')
                    print(f'Train loss: {train_loss:.4f} accuracy: {train_acc:.4f}')

                    print(f'Val loss {eval_loss:.4f} accuracy {eval_acc:.4f}')

                if save_model:
                    if eval_acc > best_accuracy:
                        torch.save(self.model, self.model_save_path)
                        best_accuracy = eval_acc

        return self 

    def eval(self, eval):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        eval_dl = DataLoader(eval, batch_size=32, shuffle=True)
        with torch.no_grad():
            for data in eval_dl:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())
        
        val_acc = correct_predictions.double() / len(eval)
        val_loss = np.mean(losses)
        return val_acc, val_loss
    
    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )
        
        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

обучение модели, сохранение обученной модели в файле

In [ ]:
model_runner = BertClassifier(model, tokenizer, optimizer, loss_fn).fit(train_dataset, eval_dataset, epochs=5)

on 6584: ===========Epoch 1/5===========
on 6584: Train loss: 0.4783 accuracy: 0.8084
on 6584: Val loss 0.4789 accuracy 0.8125
on 13168: ===========Epoch 2/5===========
on 13168: Train loss: 0.4051 accuracy: 0.8798
on 13168: Val loss 0.4492 accuracy 0.8470
on 19752: ===========Epoch 3/5===========
on 19752: Train loss: 0.3363 accuracy: 0.9390
on 19752: Val loss 0.5303 accuracy 0.8650
on 26336: ===========Epoch 4/5===========
on 26336: Train loss: 0.2810 accuracy: 0.9715
on 26336: Val loss 0.7322 accuracy 0.8718
on 32920: ===========Epoch 5/5===========
on 32920: Train loss: 0.2378 accuracy: 0.9864
on 32920: Val loss 0.7933 accuracy 0.8721


Проверка модели.
Данные взяты из Кинопоиска, отзывы на недавно вышедшие фильмы

In [ ]:
# Отрицательный отзыв на фильм Морбиус (2022) 
header = "Morbius 2022"
review = "Ну, я даже не знаю, какой супергеройский фильм последних лет может хотя бы на толику быть на столько ужасным. Абсолютно не спасает этот плевок в сторону фан-сообщества наличие нестареющего (видимо действительно вампирского происхождения) Джареда Лето. Хотя, чего уж скрывать, он всё равно красавчик, и эта роль ему на все сто процентов подходит, НО только при наличии адекватного сценария и не отвлекающегося на перекуры и другие интересные дела режиссера"
correct = "Negative"
model_out = model_runner.predict(review)
print(f'Результат модели: This review about "{header}" is {class_names[model_out]} (in fact it is {correct})')

# Положительный Avengers: Endgame, 2019
header = "Avengers: Endgame, 2019"
review = "Есть фильмы, которые хороши не потому, что в них все идеально. Картина может иметь сотню недостатков, куча дыр и несостыковок, завышенные ожидания со стороны зрителей, но все равно цепляет и оставляет по себе очень приятное послевкусие. Именно к таким фильмам лично я для себя причисляю Мстителей. Финал'. Просматривая его третий раз в кино, я поняла, что еще раз и смогу разложить и сюжет и мотивацию героев по атомам, тем не менее, ни ненависти ни какого-то огорчения я не испытываю. Если бы меня спросили, что бы я поменяла, то определенно получилось бы эссе на несколько страничек мелкого почерка. С другой стороны, мы получили весьма зрелищное и душевное окончание многолетней саги, увидели любимых персонажей, вдоволь посмеялись и даже местами поплакали. А раз фильм был способен вызвать такую гамму эмоций, значит создатели сделали почти все правильно."
correct = "Positive"
model_out = model_runner.predict(review)
print(f'Результат модели: This review about "{header}" is {class_names[model_out]} (in fact it is {correct})')

# Отрицательный Justice League, 2017
header = "Justice League, 2017"
review = "И тут все дело в подаче. Понятно что в комиксах вселенная DC давно существует и там Бэтмен и Лига Справедливости спина к спине оберегают Землю. Но в кино мире - Бэтмен всегда был одиночкой, и соперники у него были под стать - без исключительных суперспособностей. И теперь DC пытается впихнуть героя, который в массовом сознании прослыл 'одиноким рейнджером' к героям у которых настоящие суперсилы и суперспособности. Бэтмен на их фоне выглядит ну совсем никак. Но даже не смотря на диссонанс с Бэтменом у DC большие проблемы: 'Лига справедливости', по сравнению с 'Мстителями' выглядит просто дешевой поделкой, вроде все как у них но нет того ощущения постоянного драйва, герои раскрыты однобоко, а вечный пафос раздражает - так как ощущается чем то инородным и неуместным к данной картине. Сюжет в Лиге очень плоский, спецэффекты на уровне, но если их не сдабривать нужным эмоциональным фоном, они начинают смотреться как нарезка трюков. Конечно были в фильмы проблески чего то хорошего, но на общем фоне картины они не осели в памяти, зато осело разочарование от завышенных ожиданий"
correct = "Negative"
model_out = model_runner.predict(review)
print(f'Результат модели: This review about "{header}" is {class_names[model_out]} (in fact it is {correct})')

Результат модели: This review about "Morbius 2022" is Neutral (in fact it is Negative)
Результат модели: This review about "Avengers: Endgame, 2019" is Positive (in fact it is Positive)
Результат модели: This review about "Justice League, 2017" is Negative (in fact it is Negative)
